## CURRENT NOTES

There are no functions yet

Let's work this week to implement the determinisitic model sent to us

In [1]:
## This code establishes the bacteria class to be used in the agent-based model
## The final model does not have to be done in this notebook
## This should just be the building ground for the individuals in the ABM

In [1]:
import networkx as nx
import pandas as pd
from matplotlib import pyplot as plt
import datetime
import numpy as np 
import math
import pylab as pl
import math

In [2]:
##initializing the bacteria class

class bacteria:
    
    def __init__(self,carrier,tax,gene_pres,gene,mge_pres,mge,biome):
        
        self.carrier = carrier
        self.tax = tax
        self.gene_pres = gene_pres
        self.gene = gene
        self.mge_pres = mge_pres
        self.mge = mge
        self.biome = biome
        

In [3]:
##Testing the class
##This does not need to be kept in the code
##This is an example that's not necessarily true

strep = bacteria(1,'streptococcus',1,'ArB1',0,0,'ocean')
strep.gene

'ArB1'